In [2]:
# Generic imports
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import PIL


# Local imports
import sys
sys.path.append('../../../../')
import stlstuff as sls
import imagestuff as ims
import facetbrightnessstuff9 as fbs 
import f90nml

In [3]:
pwd

'/Users/chemistry/Desktop/Lincoln2018/Precious/Crystals/2018-07-05 /Case1.0 (calibration)'

In [4]:
%matplotlib notebook

In [5]:
# Parameters
ABCDangle_deg = 15.0
theta = ABCDangle_deg*np.pi/180

In [6]:
# Specifying the boxes/vectors and output file
Boxesfile = 'Boxes.nml'
Boxes=f90nml.read(Boxesfile) #reads the file at a given path
Calibrationfile = 'Calibration.nml'

In [7]:
# Define boxes for calibration
nx1list=Boxes['Boxes']['nx1list']
ny1list=Boxes['Boxes']['ny1list']
labellist=Boxes['Boxes']['labellist']
boxsize=Boxes['Boxes']['boxsize']; print (boxsize)

# Packaging these values for subsequent use
nboxes = len(nx1list); print ("nboxes =", nboxes)
nx2list = np.array(nx1list)+boxsize; print(nx2list)
ny2list = np.array(ny1list)+boxsize; print(ny2list)

20
nboxes = 3
[460 363 360]
[255 347 255]


In [8]:
# Define vectors for calibration
xorigin=Boxes['Vectors']['xorigin']
yorigin=Boxes['Vectors']['yorigin']
xa=Boxes['Vectors']['xa']
ya=Boxes['Vectors']['ya']
xb=Boxes['Vectors']['xb']
yb=Boxes['Vectors']['yb']
xc=Boxes['Vectors']['xc']
yc=Boxes['Vectors']['yc']

# Packaging these values for subsequent use
linea = [xorigin,yorigin,xa,ya]; print (linea)
lineb = [xorigin,yorigin,xb,yb]
linec = [xorigin,yorigin,xc,yc]

[472, 280, 460, 53]


In [9]:
# Read the input dataset, including spacing in micrometers
imageroot = '2018-06-28_1.1'
dx,dy,cA,cB,cC,cD,Filename = ims.getc2('', 'SEMimages/', imageroot)
print("Filename, dx and dy", Filename, dx, dy)

FileNotFoundError: [Errno 2] No such file or directory: 'SEMimages/2018-06-28_1.1-A.bmp'

In [10]:
# Graph the segments
im = PIL.Image.open(Filename)
draw = PIL.ImageDraw.Draw(im)
draw.line(linea, fill=200,width=5)
draw.line(lineb, fill=150,width=5)
draw.line(linec, fill=100,width=5)
draw.text((xa-20,ya-20), 'a')
draw.text((xb-20,yb+20), 'b')   
for i in range(nboxes):
    nx1 = nx1list[i]
    nx2 = nx2list[i]
    ny1 = ny1list[i]
    ny2 = ny2list[i]
    ims.myrectanglelabel(draw,(nx1,ny1),(nx2,ny2),labellist[i])
plt.figure()    
plt.title(imageroot)
plt.show(im,cmap = 'Greys_r', vmin = 0,vmax = 255)
im.show() # This shouldn't be necessary but there seems to be a bug with plt.show

NameError: name 'Filename' is not defined

In [11]:
# Calculate normal vectors of the crystal ... Details of this will change for each crystal
avec, bvec, cvec, navec, nbvec = fbs.solveforabc(\
                 xa-xorigin,ya-yorigin,\
                 xb-xorigin,yb-yorigin,\
                 xc-xorigin,yc-yorigin)
print ('cvec =\n',cvec)
print ('avec =\n',avec)
print ('bvec =\n',bvec)
print ('navec =\n',navec)
print ('nbvec =\n',nbvec)

Rot28 = ims.myrotation_matrix(avec, -28-90.) # use avec or bvec depending on which one the pyramidal facet touches
navec = Rot28*navec # solverforabc gives this as already normalized 
print ('unit normal a-facet =\n',navec)

Rot28 = ims.myrotation_matrix(bvec, 28+90.) # use avec or bvec depending on which one the pyramidal facet touches
nbvec = Rot28*nbvec # solverforabc gives this as already normalized 
print ('unit normal b-facet =\n',nbvec)

ndvec = -cvec # solverforabc gives this as already normalized 
print ('unit normal d-facet =\n',ndvec)

found 16 solutions
physically reasonable solution is # 7
cvec =
 [[ 0.16415094]
 [ 0.0105225 ]
 [-0.98637911]]
avec =
 [[-0.05277976]
 [-0.99841705]
 [-0.0194344 ]]
bvec =
 [[-0.82666439]
 [ 0.54705731]
 [-0.13173568]]
navec =
 [[-0.98502222]
 [ 0.05525102]
 [-0.16333572]]
nbvec =
 [[-0.53821972]
 [-0.83702902]
 [-0.09849854]]
unit normal a-facet =
 [[ 0.31750325]
 [-0.0352296 ]
 [ 0.94760253]]
unit normal b-facet =
 [[0.10774218]
 [0.38367051]
 [0.91716332]]
unit normal d-facet =
 [[-0.16415094]
 [-0.0105225 ]
 [ 0.98637911]]


In [12]:
# This is just checking
im = PIL.Image.open(Filename)
fig, ax = plt.subplots()
draw = PIL.ImageDraw.Draw(im)
for i in range(nboxes):
    nx1 = nx1list[i]
    nx2 = nx2list[i]
    ny1 = ny1list[i]
    ny2 = ny2list[i]
    ims.myrectanglelabel(draw,(nx1,ny1),(nx2,ny2),labellist[i])
ax.set_title(imageroot)

# For boxes "a", "b", and "d"
amp = 100
boxcenterx = nx1list[0]+boxsize/2
boxcentery = ny1list[0]+boxsize/2
linea_disp = list(np.squeeze([boxcenterx,boxcentery,boxcenterx+navec[0]*amp,boxcentery+navec[1]*amp]).astype(int))
draw.line(linea_disp, fill=200,width=3)

boxcenterx = nx1list[1]+boxsize/2
boxcentery = ny1list[1]+boxsize/2
lineb_disp = list(np.squeeze([boxcenterx,boxcentery,boxcenterx+nbvec[0]*amp,boxcentery+nbvec[1]*amp]).astype(int))
draw.line(lineb_disp, fill=200,width=3)

boxcenterx = nx1list[2]+boxsize/2
boxcentery = ny1list[2]+boxsize/2
lineb_disp = list(np.squeeze([boxcenterx,boxcentery,boxcenterx+ndvec[0]*amp,boxcentery+ndvec[1]*amp]).astype(int))
draw.line(lineb_disp, fill=200,width=3)

plt.show(im,cmap = 'Greys_r', vmin = 0,vmax = 255)
im.show() # This shouldn't be necessary but there seems to be a bug with plt.show

NameError: name 'Filename' is not defined

In [ ]:
# For each detector, get s-values
sAa, sBa, sCa, sDa = ims.mygets(navec,theta)
sAb, sBb, sCb, sDb = ims.mygets(nbvec,theta)
sAd, sBd, sCd, sDd = ims.mygets(ndvec,theta)
slistA = np.squeeze(np.array([sAa, sAb, sAd]))
slistB = np.squeeze(np.array([sBa, sBb, sBd]))
slistC = np.squeeze(np.array([sCa, sCb, sCd]))
slistD = np.squeeze(np.array([sDa, sDb, sDd]))

# For each detector, get s-values
sAa, sBa, sCa, sDa = ims.mygets(navec,theta)
sAb, sBb, sCb, sDb = ims.mygets(nbvec,theta)
sAd, sBd, sCd, sDd = ims.mygets(ndvec,theta)
slistA = np.squeeze(np.array([sAa, sAb, sAd]))
slistB = np.squeeze(np.array([sBa, sBb, sBd]))
slistC = np.squeeze(np.array([sCa, sCb, sCd]))
slistD = np.squeeze(np.array([sDa, sDb, sDd]))

# Extract the observed intensities
cA_obs = []
cB_obs = []
cC_obs = []
cD_obs = []
for isegment in range(nboxes):
    nx1=nx1list[isegment]; nx2=nx2list[isegment] 
    ny1=ny1list[isegment]; ny2=ny2list[isegment]
    cA_obs.append(np.mean(cA[ny1:ny2,nx1:nx2].astype('float')))
    cB_obs.append(np.mean(cB[ny1:ny2,nx1:nx2].astype('float')))
    cC_obs.append(np.mean(cC[ny1:ny2,nx1:nx2].astype('float')))
    cD_obs.append(np.mean(cD[ny1:ny2,nx1:nx2].astype('float')))

In [ ]:
# See what the A-D detector parameters look like graphically
plt.figure()
markersize = 10
plt.plot(slistA,cA_obs,'or',markersize=15)
plt.plot(slistB,cB_obs,'<b',markersize=15)
plt.plot(slistC,cC_obs,'sy',markersize=15)
plt.plot(slistD,cD_obs,'>g',markersize=15)
plt.legend(['A', 'B', 'C', 'D'],loc='upper left')
plt.plot(slistA[0],cA_obs[0],'k*')
plt.plot(slistA[1],cA_obs[1],'kx')
plt.plot(slistA[2],cA_obs[2],'k+')
plt.plot(slistB[0],cB_obs[0],'k*')
plt.plot(slistB[1],cB_obs[1],'kx')
plt.plot(slistB[2],cB_obs[2],'k+')
plt.plot(slistC[0],cC_obs[0],'k*')
plt.plot(slistC[1],cC_obs[1],'kx')
plt.plot(slistC[2],cC_obs[2],'k+')
plt.plot(slistD[0],cD_obs[0],'k*')
plt.plot(slistD[1],cD_obs[1],'kx')
plt.plot(slistD[2],cD_obs[2],'k+')
plt.grid()
srange = [-.6,.6]
plt.xlim(srange)
plt.xlabel('$s$')
plt.ylabel('$c_{obs}$')


# Fitting
maxorder = 1
order = min(len(slistA)-1,maxorder)
pA = np.polyfit(slistA,cA_obs,order); print('pA =', pA[0], ',', pA[1])
pB = np.polyfit(slistB,cB_obs,order); print('pB =', pB[0], ',', pB[1])
pC = np.polyfit(slistC,cC_obs,order); print('pC =', pC[0], ',', pC[1])
pD = np.polyfit(slistD,cD_obs,order); print('pD =', pD[0], ',', pD[1])
s_theory = np.linspace(srange[0],srange[1])
cA_theory = np.polyval(pA,s_theory)
cB_theory = np.polyval(pB,s_theory)
cC_theory = np.polyval(pC,s_theory)
cD_theory = np.polyval(pD,s_theory)
plt.plot(s_theory,cA_theory,'-r',linewidth=2)
plt.plot(s_theory,cB_theory,'--b',linewidth=2)
plt.plot(s_theory,cC_theory,'-.y',linewidth=2)
plt.plot(s_theory,cD_theory,':g',linewidth=2)
plt.title(imageroot)

In [ ]:
# Save the calibration file
cfile = open(Calibrationfile,'w')
cfile.write('&Calibration\n')
cfile.write('   '+'pA = '+str(pA[0])+',  '+str(pA[1])+'\n')
cfile.write('   '+'pB = '+str(pB[0])+',  '+str(pB[1])+'\n')
cfile.write('   '+'pC = '+str(pC[0])+',  '+str(pC[1])+'\n')
cfile.write('   '+'pD = '+str(pD[0])+',  '+str(pD[1])+'\n')
cfile.write('/ \n')
cfile.close()